In [2]:
from gen_ai_hub.proxy.langchain.amazon import ChatBedrock
from gen_ai_hub.proxy.langchain.openai import ChatOpenAI
llm = ChatBedrock(model_name="anthropic--claude-3.5-sonnet", temperature=0)
openai_llm = ChatOpenAI(proxy_model_name="gpt-4o", temperature=0.0)

# Email Classification Example

In [3]:
from langchain.prompts import PromptTemplate

email_classification_prompt = PromptTemplate.from_template(
""""You are an email classifier for a large energy company in Austria.

Assess which type of inquiry the email refers to: {categories}.

Also assess the urgency of the request as "low", "medium", "high", or "critical".

If available in the email, also extract the senders name and their phone number. Otherwise, do not add it to the JSON Object. If the phone number contains a country code, write it down as starting with plus, followed by the country code. Do not add any spaces between numbers.

Also summarize the given email as shortly as possible and in English. Focus on the issues described and actions to take. Add the summary as "shortSummary" to the JSON.

Respond only with a raw JSON Object in the following format, do not add any formatting:
{{"type": string, "urgency": string, "senderName": string, "phoneNumber": string, "shortSummary": string}}

---- Email ----

{email_content}""")

email_classification_runnable = email_classification_prompt | llm

In [4]:
email_content = """Hallo,

ich bin daran interessiert, mehr über die Möglichkeiten für nachhaltige Kraftstoffe zu erfahren, die Ihr Unternehmen anbietet. Besonders interessiert mich, welche Arten von nachhaltigen Kraftstoffen angeboten werden und wie diese zur Reduzierung des CO2-Ausstoßes beitragen können.

Wäre es möglich, detaillierte Informationen über die Eigenschaften dieser Kraftstoffe und deren Verfügbarkeit zu erhalten?

Falls Sie noch weitere Informationen benötigen, erreichen Sie mich unter +43123456789.

Vielen Dank für Ihre Unterstützung.

Mit freundlichen Grüßen,
Johann Moser
XYZ Transport GmbH
"""

categories = ["naturalGasInquiry", "classicalFuelInquiry", "hydrogenInquiry", "sustainableFuelInquiry", "partnershipProposal", "technicalSupport", "feedback", "billingIssue", "generalInquiry", "other"]

res = email_classification_runnable.invoke({"categories": categories, "email_content": email_content})
print(res)

content='{"type": "sustainableFuelInquiry", "urgency": "low", "senderName": "Johann Moser", "phoneNumber": "+43123456789", "shortSummary": "Customer requests information about sustainable fuels offered, their CO2 reduction potential, properties, and availability."}' additional_kwargs={'usage': {'prompt_tokens': 0, 'completion_tokens': 0, 'total_tokens': 0}, 'stop_reason': 'end_turn', 'model_id': 'anthropic.claude-3-5-sonnet-20240620-v1:0'} response_metadata={'usage': {'prompt_tokens': 0, 'completion_tokens': 0, 'total_tokens': 0}, 'stop_reason': 'end_turn', 'model_id': 'anthropic.claude-3-5-sonnet-20240620-v1:0'} id='run-295a4cd3-6bf4-4dcc-8aed-7384d25b46a3-0'


In [5]:
import json
jsonData = json.loads(res.content)
print("Type of request: "+jsonData.get("type","Not given"))
print("Urgency of request: "+str(jsonData.get("urgency","Not given")))
print("Sender Name: "+str(jsonData.get("senderName","Not given")))
print("Phone Number: "+str(jsonData.get("phoneNumber","Not given")))
print("Summary: "+str(jsonData.get("shortSummary","Not given")))

Type of request: sustainableFuelInquiry
Urgency of request: low
Sender Name: Johann Moser
Phone Number: +43123456789
Summary: Customer requests information about sustainable fuels offered, their CO2 reduction potential, properties, and availability.


.

.

.

.

.

.

.

.

.


# AI Agent Example

In [6]:
from langchain_core.prompts import PromptTemplate
react_agent_prompt = PromptTemplate.from_template(
    """You are an AI Agent ("Assistant") designed to be able to assist with a wide range of tasks, from answering simple questions to providing in-depth explanations and discussions on a wide range of topics. As a language model, Assistant is able to generate human-like text based on the input it receives, allowing it to engage in natural-sounding conversations and provide responses that are coherent and relevant to the topic at hand.
    
    Assistant is constantly learning and improving, and its capabilities are constantly evolving. It is able to process and understand large amounts of text, and can use this knowledge to provide accurate and informative responses to a wide range of questions. Additionally, Assistant is able to generate its own text based on the input it receives, allowing it to engage in discussions and provide explanations and descriptions on a wide range of topics.
    
    Overall, Assistant is a powerful tool that can help with a wide range of tasks and provide valuable insights and information on a wide range of topics. Whether you need help with a specific question or just want to have a conversation about a particular topic, Assistant is here to assist.
    
    TOOLS:
    ------
    
    Assistant has access to the following tools:
    
    {tools}
    
    To use a tool, please use the following format:
    
    ```
    Thought: Do I need to use a tool? Yes
    Action: the action to take, should be one of [{tool_names}]
    Action Input: the input to the action\nObservation: the result of the action
    ```
    
    When you have a response to say to the Human, or if you do not need to use a tool, you MUST use the format:
    
    ```
    Thought: Do I need to use a tool? No
    Final Answer: [your response here]
    ```
    
    Begin!
    
    Previous conversation history:
    {chat_history}
    
    New input: {input}
    {agent_scratchpad}"""
)

In [7]:
from langchain.memory import ConversationBufferMemory
# from langchain import hub
from langchain.agents import load_tools, AgentExecutor, create_react_agent

# prompt = hub.pull("hwchase17/react-chat")

tools = load_tools(
    ["llm-math"],
    llm=openai_llm
)

memory = ConversationBufferMemory(memory_key="chat_history")

conversational_agent = create_react_agent(
    tools=tools,
    llm=openai_llm,
    prompt=react_agent_prompt
)

agent_executor = AgentExecutor(
    agent=conversational_agent,
    tools=tools,
    handle_parsing_errors=True,
    verbose=True,
    memory=memory,
    max_interations=5
)

In [8]:
result = agent_executor.invoke({"input": 
    "An energy supplier company needs to calculate the expected monthly electricity usage next year, assuming it increases by 5,75% year-over-year and is evenly distributed across all months. The electricity meter reading on 01.01.2023 was 1056735 kWh and on 01.01.2024 it was 1098332 kWh."
})

print(result["output"])



> Entering new AgentExecutor chain...
To calculate the expected monthly electricity usage for next year, we need to follow these steps:

1. **Determine the total electricity usage for the current year (2023):**
   - Subtract the meter reading on 01.01.2023 from the meter reading on 01.01.2024.

2. **Calculate the expected total electricity usage for the next year (2024):**
   - Increase the total electricity usage for 2023 by 5.75%.

3. **Calculate the expected monthly electricity usage for 2024:**
   - Divide the expected total electricity usage for 2024 by 12.

Let's go through these steps one by one.

### Step 1: Determine the total electricity usage for 2023
```
Total usage in 2023 = Meter reading on 01.01.2024 - Meter reading on 01.01.2023
```
```
Total usage in 2023 = 1098332 kWh - 1056735 kWh
```

Thought: Do I need to use a tool? Yes
Action: Calculator
Action Input: 1098332 - 1056735
Answer: 41597### Step 2: Calculate the expected total electricity usage for the next year (20